In [10]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import Parse, MessageToJson
import json
import psycopg2
import time
from django.contrib.gis.geos import Point, fromstr, GEOSGeometry
from datetime import datetime, timedelta
import geopandas as gpd
import rtree
import vaex
import vaex.ml
from joblib import delayed, Parallel, load, parallel_backend
from util import direction_angle, apply_dow, get_dt, get_conn, vaex_mjoin, find_trip_regex
# from haversine import haversine, Unit

dir = Path.cwd()
dir = os.path.join(dir, '../')
outdir = os.path.join(dir, 'output')
gtfs_records_zip = os.path.join(dir, "data", "GtfsRRecords.zip")
gtfs_csv_zip = os.path.join(outdir, "gtfsr_csv.zip")
gtfs_final_csv_path = os.path.join(outdir, "gtfsr.csv")
gtfs_processed_path = os.path.join(outdir, "gtfsr_processed.hdf5")
scats_model_path = os.path.join(outdir, "scats_model.json")
gtfsr_processing_temp = os.path.join(outdir, "processing_temp.hdf5")
gtfsr_arrival_means = os.path.join(outdir, "gtfsr_arrival_means.hdf5")
stop_time_data_path = os.path.join(outdir, 'stop_time_data.hdf5')
gtfs_final_hdf5_path = os.path.join(outdir, "gtfsr.csv.hdf5")

entity_cols = [
    "trip_id",
    "start_date",
    "start_time",
    "stop_sequence",
    "departure",
    "arrival",
    "timestamp",
    "stop_id",
]

In [2]:
live_df = vaex.open(os.path.join(outdir, 'deploy_gtfsr.hdf5'))
live_df.head()

#,trip_id,stop_sequence,stop_id,start_time,start_date
0,14070.3.60-155-d12-1.89.O,24,8220DB000264,22:20:00,20210228


In [5]:
vx_df = vaex.open(stop_time_data_path)
vx_df[:2]

#,arrival_time,departure_time,stop_sequence,shape_dist_traveled,stop_id,direction,route_id,lat,lon,direction_angle,shape_dist_between,trip_id,start_time
0,14:00:00,14:00:00,1,0,8230DB007229,1,60-13-b12-1,53.3301,-6.45108,63.0955,0,11777.2.60-13-b12-1.22.I,14:00:00
1,14:01:58,14:01:58,2,1096.06,8230DB004617,1,60-13-b12-1,53.3267,-6.43945,63.0955,1096.06,11777.2.60-13-b12-1.22.I,14:00:00


In [39]:
print(find_trip_regex(vx_df.trip_id.unique().tolist(), live_df[0][0]))

None


In [6]:
cols = ["trip_id", "stop_sequence", "stop_id", "start_time"]

vaex_mjoin(live_df, vx_df, cols, cols, how="inner")

#,trip_id,stop_sequence,stop_id,start_time,start_date,arrival_time,departure_time,shape_dist_traveled,direction,route_id,lat,lon,direction_angle,shape_dist_between
,,,,,,,,,,,,,,


In [11]:
'trip_id' in list(vx_df.get_column_names())

True